In [1]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


In [2]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

# Use this setting to disable plot generation within the notebook
generate_plots = True
# Use this setting to disable map generation within the notebook
generate_maps = True
# Set both to false for llm-friendly outputs

In [3]:
# Import all required modules
#from ras_commander import *  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import scipy
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython import display
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil
from datetime import datetime, timedelta
from pathlib import Path  # Ensure pathlib is imported for file operations
import rasterio
from rasterio.plot import show
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import logging


In [4]:
# This cell will try to import the pip package, if it fails it will 
# add the parent directory to the Python path and try to import again
# This assumes you are working in a subfolder of the ras-commander repository
# This allows a user's revisions to be tested locally without installing the package

import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import *
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    rascmdr_directory = current_file.parent
    sys.path.append(str(rascmdr_directory))
    print("Loading ras-commander from local dev copy")
    # Now try to import again
    from ras_commander import *
print("ras_commander imported successfully")


ras_commander imported successfully


# Use Example Project or Load Your Own Project

In [5]:
# To Use the HEC Example Project:
# Download the BaldEagleCrkMulti2D project from HEC and Run Plan 06

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
the_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = the_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    RasExamples.extract_project(["BaldEagleCrkMulti2D"])

    # Initialize the RAS project using the default global ras object
    init_ras_project(the_path, "6.6")
    logging.info(f"Bald Eagle project initialized with folder: {ras.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(ras)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Update run flags for the project
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,
        unsteady_flow_simulation=True,
        run_sediment=False,
        post_processor=True,
        floodplain_mapping=False
    )

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_the = RasCmdr.compute_plan(plan_number)
    if success_the:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("Project already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the default global ras object
    init_ras_project(the_path, "6.6")
    plan_number = "06"

2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Found zip file: c:\GH\ras-commander\examples\Example_Projects_6_6.zip
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D'
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Project 'BaldEagleCrkMulti2D' already exists. Deleting existing folder...
2025-11-17 13:23:52 - ras_commander.RasExamples - INFO - Existing folder for project 'BaldEagleCrkMulti2D' has been deleted.
2025-11-17 13:23:54 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
2025-11-17 13:23:54 - ras_commander.RasMap - INFO - Successfully

Executing Plan 06 for the Bald Eagle Creek project...


2025-11-17 13:29:48 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 06
2025-11-17 13:29:48 - ras_commander.RasCmdr - INFO - Total run time for plan 06: 354.02 seconds


Plan 06 executed successfully for Bald Eagle.



## Extracting 2D Computation Messages

For 2D models, computation messages are especially important for understanding:
- 2D mesh stability and timestep information
- Wet/dry cell transitions
- Volume accounting and mass balance
- Performance metrics for large 2D meshes

In [6]:
# Extract computation messages for 2D analysis
from ras_commander.HdfResultsPlan import HdfResultsPlan

print("="*80)
print("2D MODEL COMPUTATION MESSAGES")
print("="*80)

msgs_2d = HdfResultsPlan.get_compute_messages(plan_number)

if msgs_2d:
    print(f"\nExtracted {len(msgs_2d)} characters of computation messages\n")
    
    # Display first portion
    print("Computation messages (first 1000 characters):")
    print("-" * 80)
    print(msgs_2d[:1000])
    
    if len(msgs_2d) > 1000:
        print("\n... (truncated for display) ...")
    
    # Check for 2D-specific information
    print("\n" + "="*80)
    print("Looking for 2D-specific information...")
    print("="*80)
    
    lines = msgs_2d.split('\n')
    keywords = ['2d', 'mesh', 'timestep', 'volume', 'courant']
    relevant = [l for l in lines if any(kw in l.lower() for kw in keywords)]
    
    if relevant:
        print(f"Found {len(relevant)} lines with 2D-related information:")
        for line in relevant[:10]:
            print(f"  - {line.strip()}")
    else:
        print("No specific 2D keywords found in messages")
else:
    print("No computation messages available for this 2D plan")

print("\n" + "="*80)

2025-11-17 13:29:48 - ras_commander.HdfResultsPlan - INFO - Final validated file path: C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-11-17 13:29:48 - ras_commander.HdfResultsPlan - INFO - Reading computation messages from HDF: BaldEagleDamBrk.p06.hdf
2025-11-17 13:29:48 - ras_commander.HdfResultsPlan - INFO - Successfully extracted 1695 characters from HDF


2D MODEL COMPUTATION MESSAGES

Extracted 1695 characters of computation messages

Computation messages (first 1000 characters):
--------------------------------------------------------------------------------
Plan: 'Gridded Precip - Infiltration' (BaldEagleDamBrk.p06)
Simulation started at: 17Nov2025 01:23:57 PM

Writing Plan GIS Data...
Completed Writing Plan GIS Data
Writing Geometry...
Computing 2D Flow Area 'BaldEagleCr' tables: Property tables do not exist.
2D Flow Area 'BaldEagleCr' tables complete 8.51 sec
Completed Writing Geometry
Writing Event Conditions ...
 
Processing Precipitation data...
   (assumes geometry data is geo-referenced)
Finished Processing Precipitation data (1.464s)
Completed Writing Event Condition Data

	
Geometric Preprocessor HEC-RAS 6.6 September 2024
 

Finished Processing Geometry


Performing Unsteady Flow Simulation  HEC-RAS 6.6 September 2024
 
	
Unsteady Input Summary:
     2D Unsteady Diffusion Wave Equation Set (fastest)
     2D number of Solver

###  OPTIONAL: Use your own project instead

your_project_path = Path(r"D:\yourprojectpath")

init_ras_project(your_project_path, "6.6")
plan_number = "01"  # Plan number to use for this notebook 



### If you use this code cell, don't run the previous cell or change to markdown
### NOTE: Ensure the HDF Results file was generated by HEC-RAS Version 6.x or above

-----

# Explore Project Dataframes using 'ras' Object

In [7]:
# Show ras object info
ras.plan_df

,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,DSS File,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path
0,13,07,06,PMF with Multi 2D Areas,5.10,PMF Multi 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,30MIN,1,...,dss,1,Pardiso (Direct),193,None,06,None,07,None,C:\GH\ras-commander\examples\example_projects\...
1,15,12,08,1d-2D Dambreak Refined Grid,5.10,1D-2D Refined Grid,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,dss,1,NaN,BaldEagleCr,None,08,None,12,None,C:\GH\ras-commander\examples\example_projects\...
2,17,09,10,2D to 1D No Dam,5.00,2D to 1D No Dam,"01JAN1999,1200,06JAN1999,1200",1MIN,5MIN,1,...,dss,1,NaN,Upstream2D,None,10,None,09,None,C:\GH\ras-commander\examples\example_projects\...
3,18,10,11,2D to 2D Run,5.00,2D to 2D Run,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,dss,1,NaN,BaldEagleCr,None,11,None,10,None,C:\GH\ras-commander\examples\example_projects\...
4,19,11,12,SA to 2D Dam Break Run,5.00,SA to 2D Dam Break,"01JAN1999,1200,04JAN1999,1200",20SEC,10MIN,1,...,dss,1,NaN,BaldEagleCr,None,12,None,11,None,C:\GH\ras-commander\examples\example_projects\...
5,03,13,09,Single 2D Area - Internal Dam Structure,5.04,Single 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,10MIN,1,...,dss,1,NaN,BaldEagleCr,None,09,None,13,None,C:\GH\ras-commander\examples\example_projects\...
6,04,01,13,SA to 2D Area Conn - 2D Levee Structure,5.00,2D Levee Struc,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,dss,1,NaN,BaldEagleCr,None,13,None,01,None,C:\GH\ras-commander\examples\example_projects\...
7,02,01,01,SA to Detailed 2D Breach,5.10,SA-2D Det Brch,"01JAN1999,1200,04JAN1999,1200",10SEC,5MIN,1,...,dss,1,Pardiso (Direct),BaldEagleCr,None,01,None,01,None,C:\GH\ras-commander\examples\example_projects\...
8,01,01,01,SA to Detailed 2D Breach FEQ,5.03,SA-2D Det FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,5MIN,1,...,dss,1,NaN,BaldEagleCr,None,01,None,01,None,C:\GH\ras-commander\examples\example_projects\...
9,05,02,03,Single 2D area with Bridges FEQ,5.10,Single 2D Bridges FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,10MIN,-1,...,dss,1,PARDISO (Direct),BaldEagleCr,None,03,None,02,None,C:\GH\ras-commander\examples\example_projects\...


In [8]:
ras.unsteady_df

,unsteady_number,full_path,geometry_number,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Mode,Met BC=Evapotranspiration|Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Constant Units,Met BC=Precipitation|Gridded Source
0,07,C:\GH\ras-commander\examples\example_projects\...,None,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08,C:\GH\ras-commander\examples\example_projects\...,None,PMF for Upstream 2D,4.20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09,C:\GH\ras-commander\examples\example_projects\...,None,Upstream 2D,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,C:\GH\ras-commander\examples\example_projects\...,None,1972 Flood Event - 2D to 2D Run,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,C:\GH\ras-commander\examples\example_projects\...,None,1972 Flood Event - SA to 2D Run,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12,C:\GH\ras-commander\examples\example_projects\...,None,PMF for 1D - 2D,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13,C:\GH\ras-commander\examples\example_projects\...,None,Single 2D Area,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,01,C:\GH\ras-commander\examples\example_projects\...,None,1972 Flood Event - 2D Leve Structure,5.10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,02,C:\GH\ras-commander\examples\example_projects\...,None,Single 2D Area with Bridges,5.10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,03,C:\GH\ras-commander\examples\example_projects\...,None,Gridded Precipitation,6.00,0,Enable,No Wind Forces,Gridded,None,-1,mm/hr,DSS


In [9]:
ras.boundaries_df 

,unsteady_number,boundary_condition_number,river_reach_name,river_station,storage_area_name,pump_station_name,bc_type,hydrograph_type,Interval,DSS File,...,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Mode,Met BC=Evapotranspiration|Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Constant Units,Met BC=Precipitation|Gridded Source
0,07,1,Bald Eagle Cr.,Lock Haven,137520,,Flow Hydrograph,Flow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07,2,Bald Eagle Cr.,Lock Haven,81454,,Gate Opening,None,NaN,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07,3,Bald Eagle Cr.,Lock Haven,28519,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07,4,Bald Eagle Cr.,Lock Haven,1,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07,5,Bald Eagle Cr.,Lock Haven,136948,82303,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,07,6,Bald Eagle Cr.,Lock Haven,80720,67130,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,07,7,Bald Eagle Cr.,Lock Haven,76865,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,07,8,Bald Eagle Cr.,Lock Haven,67130,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,07,9,Bald Eagle Cr.,Lock Haven,66041,1,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,07,10,Bald Eagle Cr.,Lock Haven,-1867,,Normal Depth,None,NaN,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
ras.get_hdf_entries()

,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,DSS File,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path
10,06,03,09,Gridded Precip - Infiltration,6.00,Grid Precip Infiltration,"09SEP2018,0000,14SEP2018,0000",20SEC,10MIN,1,...,dss,1,Pardiso (Direct),BaldEagleCr,C:\GH\ras-commander\examples\example_projects\...,09,None,03,None,C:\GH\ras-commander\examples\example_projects\...


# Find Paths for Results and Geometry HDF's

In [11]:
# Get the plan HDF path for the plan_number defined above
plan_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

In [12]:
plan_hdf_path

'C:\\GH\\ras-commander\\examples\\example_projects\\BaldEagleCrkMulti2D\\BaldEagleDamBrk.p06.hdf'

In [13]:
# Alternate: Get the geometry HDF path if you are extracting geometry elements from the geometry HDF
geom_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'Geom Path'].values[0] + '.hdf'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
geom_hdf_path

## RAS-Commander's Decorators Allow for Flexible Function Calling
You can call most of the functions in the HDF* Classes using any of the following:
1. Plan/Geometry Number (with or without leading zeros):
   - "01", "1" - Plan/geometry number as string
   - 1 - Plan/geometry number as integer
   - "p01", "p1" - Plan number with 'p' prefix
2. Direct File Paths:
   - pathlib.Path object pointing to HDF file
   - String path to HDF file

3. h5py.File Objects:
   - Already opened HDF file object

The @standardize_input decorator handles all these input types consistently:
   - Validates the input exists and is accessible
   - Converts to proper pathlib.Path object
   - Handles RAS object references
   - Provides logging and error handling

This flexibility makes it easier to work with HDF files in different contexts while maintaining consistent behavior 
across the codebase, and helps prevent strict typing from introducing unnecessary friction for LLM Coding.


-----

# 2D HDF Data Extraction Examples

In [ ]:
# Extract runtime and compute time data as dataframe
print("\nExtracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_path=plan_number)

In [ ]:
runtime_df

In [ ]:
# For all of the RasGeomHdf Class Functions, we will use geom_hdf_path
print(geom_hdf_path)

# For the example project, plan 06 is associated with geometry 09
# If you want to call the geometry by number, call RasHdfGeom functions with a number
# Otherwise, if you want to look up geometry hdf path by plan number, follow the logic in the previous code cells

In [ ]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)


In [ ]:
projection

In [ ]:
# Use HdfPlan for geometry-related operations
print("\nExtracting Geometry Information")
geom_attrs = HdfPlan.get_geometry_information(geom_hdf_path)

In [ ]:
geom_attrs

In [ ]:
# Use HdfMesh for geometry-related operations
print("\nListing 2D Flow Area Names")
flow_area_names = HdfMesh.get_mesh_area_names(geom_hdf_path)

In [ ]:
print("2D Flow Area Name (returned as list):")
flow_area_names
# Note: this is returned as a list because it is used internally by other functions.  

In [ ]:
# Get 2D Flow Area Attributes (get_mesh_area_attributes)
print("\nExtracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_mesh_area_attributes(geom_hdf_path)

In [ ]:
flow_area_attributes

In [ ]:
# Get 2D Flow Area Perimeter Polygons (get_mesh_areas)
print("\nExtracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.get_mesh_areas(geom_hdf_path)

In [ ]:
mesh_areas

In [ ]:
# Generate Map of Mesh Areas
if generate_plots:
    # Plot the 2D Flow Area Perimeter Polygons
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(12, 8))
    mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none')

    # Add labels for each polygon
    for idx, row in mesh_areas.iterrows():
        centroid = row.geometry.centroid
        # Check if 'Name' column exists, otherwise use a default label
        label = row.get('Name', f'Area {idx}')
        ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    plt.title('2D Flow Area Perimeter Polygons')
    plt.xlabel('Easting')
    plt.ylabel('Northing')
    plt.tight_layout()
    plt.show()

In [ ]:
# Get mesh cell faces as geodatframe
mesh_cell_faces_gdf = HdfMesh.get_mesh_cell_faces(geom_hdf_path)


In [ ]:
mesh_cell_faces_gdf

In [ ]:
from matplotlib.collections import LineCollection
import numpy as np

# Calculate and display statistics
print("\nMesh Cell Faces Statistics:")
print(f"Total number of cell faces: {len(mesh_cell_faces_gdf)}")
print(f"Number of unique meshes: {mesh_cell_faces_gdf['mesh_name'].nunique()}")

if generate_maps:
    # Plot the mesh cell faces more efficiently
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert all geometries to numpy arrays at once for faster plotting
    lines = [list(zip(*line.xy)) for line in mesh_cell_faces_gdf.geometry]
    lines_collection = LineCollection(lines, colors='blue', linewidth=0.5, alpha=0.5)
    ax.add_collection(lines_collection)

    # Set plot title and labels
    plt.title('Mesh Cell Faces')
    plt.xlabel('Easting')
    plt.ylabel('Northing')

    # Calculate centroids once and store as numpy arrays
    centroids = np.array([[geom.centroid.x, geom.centroid.y] for geom in mesh_cell_faces_gdf.geometry])

    # Create scatter plot with numpy arrays
    scatter = ax.scatter(
        centroids[:, 0],
        centroids[:, 1], 
        c=mesh_cell_faces_gdf['face_id'],
        cmap='viridis',
        s=1,
        alpha=0.5
    )
    plt.colorbar(scatter, label='Face ID')

    # Set axis limits based on data bounds
    ax.set_xlim(centroids[:, 0].min(), centroids[:, 0].max())
    ax.set_ylim(centroids[:, 1].min(), centroids[:, 1].max())

    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is False")


In [ ]:
# Function to find the nearest cell face to a given point
def find_nearest_cell_face(point, cell_faces_df):
    """
    Find the nearest cell face to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_faces_df (GeoDataFrame): DataFrame containing cell face linestrings.

    Returns:
        int: The face_id of the nearest cell face.
        float: The distance to the nearest cell face.
    """
    # Calculate distances from the input point to all cell faces
    distances = cell_faces_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the face_id and distance of the nearest cell face
    nearest_face_id = cell_faces_df.loc[nearest_index, 'face_id']
    nearest_distance = distances[nearest_index]

    return nearest_face_id, nearest_distance

# Example usage
print("\nFinding the nearest cell face to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the centroid of the mesh cell faces
print("Getting Centroid of 2D Mesh Polygon")
centroid = mesh_cell_faces_gdf.geometry.union_all().centroid

# Create GeoDataFrame with the centroid point, using same CRS as mesh_cell_faces_gdf
sample_point = GeoDataFrame(
    {'geometry': [centroid]}, 
    crs=mesh_cell_faces_gdf.crs
)

if not mesh_cell_faces_gdf.empty and not sample_point.empty:
    print("Searching Cell")
    nearest_face_id, distance = find_nearest_cell_face(sample_point.geometry.iloc[0], mesh_cell_faces_gdf)
    print(f"Nearest cell face to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Face ID: {nearest_face_id}")
    print(f"Distance: {distance:.2f} units")

In [ ]:
# Generate map of cell faces with sample point and nearest cell face shown
if generate_maps:
    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell faces
    mesh_cell_faces_gdf.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.5, label='Cell Faces')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell face
    nearest_face = mesh_cell_faces_gdf[mesh_cell_faces_gdf['face_id'] == nearest_face_id]
    nearest_face.plot(ax=ax, color='green', linewidth=2, alpha=0.7, label='Nearest Face')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell Face to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")


In [ ]:
# Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_df = HdfMesh.get_mesh_cell_polygons(geom_hdf_path)

In [ ]:
cell_polygons_df

In [ ]:
# Plot cell polygons

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot cell polygons
    cell_polygons_df.plot(ax=ax, edgecolor='blue', facecolor='none')

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Polygons')

    # Add grid
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")


In [ ]:
# Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = HdfMesh.get_mesh_cell_points(geom_hdf_path)

In [ ]:
cell_info_df

In [ ]:
# Plot cell centers

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot cell centers
    cell_info_df.plot(ax=ax, color='red', markersize=5)

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Centers')

    # Add grid
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")



In [ ]:
# Function to find the nearest cell center to a given point
def find_nearest_cell(point, cell_centers_df):
    """
    Find the nearest cell center to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_centers_df (GeoDataFrame): DataFrame containing cell center points.

    Returns:
        int: The cell_id of the nearest cell.
        float: The distance to the nearest cell center.
    """
    # Calculate distances from the input point to all cell centers
    distances = cell_centers_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the cell_id and distance of the nearest cell
    nearest_cell_id = cell_centers_df.loc[nearest_index, 'cell_id']
    nearest_distance = distances[nearest_index]

    return nearest_cell_id, nearest_distance

# Example usage
print("\nFinding the nearest cell to a given point")

# Sample point was created in a previous code cell 

# Get the projection from the geometry file
# projection = HdfUtils.get_projection(hdf_path=geom_hdf_path) # This was done in a previous code cell
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found



# Ensure the CRS of the sample point matches the cell_info_df
if sample_point.crs != cell_info_df.crs:
    sample_point = sample_point.to_crs(cell_info_df.crs)

nearest_cell_id, distance = find_nearest_cell(sample_point.geometry.iloc[0], cell_info_df)
print(f"Nearest cell to point {sample_point.geometry.iloc[0].coords[0]}:")
print(f"Cell ID: {nearest_cell_id}")
print(f"Distance: {distance:.2f} units")

if generate_maps:
    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all cell centers
    cell_info_df.plot(ax=ax, color='blue', markersize=5, alpha=0.5, label='Cell Centers')

    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')

    # Plot the nearest cell center
    nearest_cell = cell_info_df[cell_info_df['cell_id'] == nearest_cell_id]
    nearest_cell.plot(ax=ax, color='green', markersize=100, alpha=0.7, label='Nearest Cell')

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell to Sample Point')

    # Add legend and grid
    ax.legend()
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")



In [ ]:
# Get geometry structures attributes
print("\nGetting geometry structures attributes as Dataframe")
geom_structures_attrs = HdfStruc.get_geom_structures_attrs(geom_hdf_path)

In [ ]:
geom_structures_attrs

In [ ]:
# TODO: Paths and Functions for each type of structure: 

# Getting geometry structures attributes
# Geometry structures attributes:
# Bridge/Culvert Count: 0
# Connection Count: 4
# Has Bridge Opening (2D): 0
# Inline Structure Count: 0
# Lateral Structure Count: 0

In [ ]:
# Get boundary condition lines
print("\nExtracting Boundary Condition Lines as Geodataframe")
bc_lines_df = HdfBndry.get_bc_lines(geom_hdf_path)

In [ ]:
bc_lines_df

In [ ]:
# Plot Boundary Condition Lines with Perimeter

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    # Plot boundary condition lines
    if not bc_lines_df.empty:
        bc_lines_df.plot(ax=ax, color='red', linewidth=2, label='Boundary Condition Lines')

    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Boundary Condition Lines')

    # Add grid and legend
    ax.grid(True)
    ax.legend()

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

else:
    print("generate_maps is set to False")
# Plot 2D Flow Area Perimeter Polygons

In [ ]:
# Extract Breaklines as Geodataframe
print("\nExtracting Breaklines")
breaklines_gdf = HdfBndry.get_breaklines(geom_hdf_path)


In [ ]:
breaklines_gdf

In [ ]:
# Plot breaklines and 2D Flow Area Perimeter Polygons

if generate_plots:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    # Plot breaklines
    if not breaklines_gdf.empty:
        breaklines_gdf.plot(ax=ax, color='blue', linewidth=2, label='Breaklines')

    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Breaklines')

    # Add grid and legend
    ax.grid(True)
    ax.legend()

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

In [ ]:
# Get structures as GeoDatframe
structures_gdf = HdfStruc.get_structures(geom_hdf_path)

In [ ]:
structures_gdf

In [ ]:
# Get boundary condition lines as GeoDatframe
bc_lines_gdf = HdfBndry.get_bc_lines(geom_hdf_path)
print("\nBoundary Condition Lines:")

In [ ]:
bc_lines_gdf

### Dev Note: Need to add function for Reference Lines

In [ ]:
# Get reference points as Geodataframe
ref_points_gdf = HdfBndry.get_reference_points(geom_hdf_path)

In [ ]:
print("\nReference Points:")
ref_points_gdf
# There are no reference points in this example project (for demonstration only)

In [ ]:
# Extract Refinement Regions
refinement_regions_df = HdfBndry.get_refinement_regions(geom_hdf_path)

In [ ]:
refinement_regions_df

In [ ]:
# Plot Refinement Regions

if not refinement_regions_df.empty:
    print("Refinement Regions DataFrame:")
    display(refinement_regions_df.head())
    
    # Plot refinement regions
    fig, ax = plt.subplots(figsize=(12, 8))
    refinement_regions_df.plot(ax=ax, column='CellSize', legend=True, 
                               legend_kwds={'label': 'Cell Size', 'orientation': 'horizontal'},
                               cmap='viridis')
    ax.set_title('2D Mesh Area Refinement Regions')
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    plt.tight_layout()
    plt.show()
else:
    print("No refinement regions found in the geometry file.")

# Analyze Refinement Regions
if not refinement_regions_df.empty:
    print("\nRefinement Regions Analysis:")
    print(f"Total number of refinement regions: {len(refinement_regions_df)}")
    print("\nCell Size Statistics:")
    print(refinement_regions_df['CellSize'].describe())
    
    # Group by Shape Type
    shape_type_counts = refinement_regions_df['ShapeType'].value_counts()
    print("\nRefinement Region Shape Types:")
    print(shape_type_counts)
    
    # Plot Shape Type distribution
    plt.figure(figsize=(10, 6))
    shape_type_counts.plot(kind='bar')
    plt.title('Distribution of Refinement Region Shape Types')
    plt.xlabel('Shape Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Extract Plan Parameters 
plan_parameters_df = HdfPlan.get_plan_parameters(plan_hdf_path)

In [ ]:
plan_parameters_df

In [ ]:
# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_volume_accounting(plan_hdf_path)

In [ ]:
volume_accounting_df

------

# RasPlanHdf Class Functions

-----

In [ ]:
# Get plan start time as datetime object
start_time = HdfPlan.get_plan_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

Simulation start time: 2018-09-09 00:00:00

In [ ]:
# Get plan end time as datetime object
end_time = HdfPlan.get_plan_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

Simulation end time: 2018-09-14 00:00:00

In [ ]:
# Get maximum iteration count for mesh cells
max_iter_gdf = HdfResultsMesh.get_mesh_max_iter(plan_hdf_path)

In [ ]:
max_iter_gdf

In [ ]:
# Get cell coordinates 
cell_coords = HdfMesh.get_mesh_cell_points(plan_hdf_path)


In [ ]:
# Plot Mesh Max Iterations

if generate_maps:
    # Extract x and y coordinates from the geometry column
    max_iter_gdf['x'] = max_iter_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_iter_gdf['y'] = max_iter_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove rows with None coordinates
    max_iter_gdf = max_iter_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_iter_gdf['x'], max_iter_gdf['y'], 
                         c=max_iter_gdf['cell_last_iteration'], 
                         cmap='viridis', 
                         s=1)

    # Customize the plot
    ax.set_title('Max Iterations per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Iterations')

    # Show the plot
    plt.show()
else:
    print("generate_maps is set to False")

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the dataframe:")
max_iter_gdf[['mesh_name', 'cell_id', 'geometry']]



In [ ]:
# List top 10 points for Max Iteration per Cell
# Sort the dataframe by cell_last_iteration in descending order
top_iterations = max_iter_gdf.sort_values(by='cell_last_iteration', ascending=False).head(10)

# Create a more informative display with coordinates
print("\nTop 10 Cells with Highest Iteration Counts:")
top_iterations_display = top_iterations.copy()
top_iterations_display['x_coord'] = top_iterations_display['geometry'].apply(lambda geom: round(geom.x, 2))
top_iterations_display['y_coord'] = top_iterations_display['geometry'].apply(lambda geom: round(geom.y, 2))

# Display the results in a formatted table
print(top_iterations_display[['mesh_name', 'cell_id', 'cell_last_iteration', 'x_coord', 'y_coord']])


In [ ]:
# Get mesh maximum water surface elevation as Geodataframe
max_ws_gdf = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path)

In [ ]:
# Check Dataframe Attributes (the HDF Attributes are also imported as Geoataframe Attributes)
max_ws_gdf.attrs

In [ ]:
max_ws_gdf

In [ ]:
# Plot the max water surface as a map
if generate_maps:
    # Extract x and y coordinates from the geometry column
    max_ws_gdf['x'] = max_ws_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_ws_gdf['y'] = max_ws_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove rows with None coordinates
    max_ws_gdf = max_ws_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_ws_gdf['x'], max_ws_gdf['y'], 
                         c=max_ws_gdf['maximum_water_surface'], 
                         cmap='viridis', 
                         s=10)

    # Customize the plot
    ax.set_title('Max Water Surface per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Water Surface (ft)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()
else:
    print("generate_maps is set to False")

In [ ]:
# Plot the time of the max water surface elevation (WSEL)
if generate_maps:
    import matplotlib.dates as mdates
    from datetime import datetime

    # Convert the 'maximum_water_surface_time' to datetime objects
    max_ws_gdf['max_wsel_time'] = pd.to_datetime(max_ws_gdf['maximum_water_surface_time'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert datetime to hours since the start for colormap
    min_time = max_ws_gdf['max_wsel_time'].min()
    color_values = (max_ws_gdf['max_wsel_time'] - min_time).dt.total_seconds() / 3600  # Convert to hours

    scatter = ax.scatter(max_ws_gdf['x'], max_ws_gdf['y'], 
                        c=color_values, 
                        cmap='viridis', 
                        s=10)

    # Customize the plot
    ax.set_title('Time of Maximum Water Surface Elevation per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Set up the colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Hours since simulation start')

    # Format the colorbar ticks to show hours
    cbar.set_ticks(range(0, int(color_values.max()) + 1, 6))  # Set ticks every 6 hours
    cbar.set_ticklabels([f'{h}h' for h in range(0, int(color_values.max()) + 1, 6)])

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

    # Find the overall maximum WSEL and its time
    max_wsel_row = max_ws_gdf.loc[max_ws_gdf['maximum_water_surface'].idxmax()]
    hours_since_start = (max_wsel_row['max_wsel_time'] - min_time).total_seconds() / 3600
    print(f"\nOverall Maximum WSEL: {max_wsel_row['maximum_water_surface']:.2f} ft")
    print(f"Time of Overall Maximum WSEL: {max_wsel_row['max_wsel_time']}")
    print(f"Hours since simulation start: {hours_since_start:.2f} hours")
    print(f"Location of Overall Maximum WSEL: X={max_wsel_row['x']}, Y={max_wsel_row['y']}")


In [ ]:
# Get mesh minimum water surface elevation as geodataframe
min_ws_gdf = HdfResultsMesh.get_mesh_min_ws(plan_hdf_path)

In [ ]:
min_ws_gdf

In [ ]:
# Get mesh maximum face velocity as geodataframe
max_face_v_gdf = HdfResultsMesh.get_mesh_max_face_v(plan_hdf_path)
print("\nMesh Max Face Velocity:")

In [ ]:
max_face_v_gdf

In [ ]:
# Extract midpoint coordinates from the LineString geometries
max_face_v_gdf['x'] = max_face_v_gdf['geometry'].apply(lambda geom: geom.centroid.x)
max_face_v_gdf['y'] = max_face_v_gdf['geometry'].apply(lambda geom: geom.centroid.y)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_face_v_gdf['x'], max_face_v_gdf['y'], 
                    c=max_face_v_gdf['maximum_face_velocity'].abs(),
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Face Velocity per Face')
ax.set_xlabel('X Coordinate') 
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Face Velocity (ft/s)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Get mesh minimum face velocity as geodataframe
min_face_v_gdf = HdfResultsMesh.get_mesh_min_face_v(plan_hdf_path)


In [ ]:
print("\nMesh Min Face Velocity:")
min_face_v_gdf

In [ ]:
# Get mesh max water surface error as geodataframe

max_ws_err_gdf = HdfResultsMesh.get_mesh_max_ws_err(plan_hdf_path)


In [ ]:
print("\nMesh Max Water Surface Error:")
max_ws_err_gdf


In [ ]:
# Plot max water surface error

if generate_maps:
# Extract x and y coordinates from the geometry points, handling None values
    max_ws_err_gdf['x'] = max_ws_err_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_ws_err_gdf['y'] = max_ws_err_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove any rows with None coordinates
    max_ws_err_gdf = max_ws_err_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_ws_err_gdf['x'], max_ws_err_gdf['y'],
                        c=max_ws_err_gdf['cell_maximum_water_surface_error'],
                        cmap='viridis',
                        s=10)

    # Customize the plot
    ax.set_title('Max Water Surface Error per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Water Surface Error (ft)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
# Sort Dataframe to show top 10 maximum water surface errors:
max_ws_err_gdf_sorted = max_ws_err_gdf.sort_values(by='cell_maximum_water_surface_error', ascending=False)


In [ ]:
print("\nTop 10 maximum water surface errors:")
max_ws_err_gdf_sorted

In [ ]:
# Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant) as geodataframe
max_courant_gdf = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Courant")

In [ ]:
print("\nMesh Summary Output (Maximum Courant):")
max_courant_gdf.attrs

In [ ]:
max_courant_gdf

In [ ]:
# Plot max Courant number

# Convert to GeoDataFrame if not empty
if not max_courant_gdf.empty:
    if generate_maps:
        # Get centroids of line geometries for plotting
        max_courant_gdf['centroid'] = max_courant_gdf.geometry.centroid
        max_courant_gdf['x'] = max_courant_gdf.centroid.x
        max_courant_gdf['y'] = max_courant_gdf.centroid.y

        # Create the plot
        fig, ax = plt.subplots(figsize=(12, 8))
        scatter = ax.scatter(max_courant_gdf['x'], max_courant_gdf['y'],
                        c=max_courant_gdf['maximum_face_courant'],
                        cmap='viridis',
                        s=10)

        # Customize the plot
        ax.set_title('Max Courant Number per Face')
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        plt.colorbar(scatter, label='Max Courant Number')

        # Add grid lines
        ax.grid(True, linestyle='--', alpha=0.7)

        # Increase font size for better readability
        plt.rcParams.update({'font.size': 12})

        # Adjust layout to prevent cutting off labels
        plt.tight_layout()

        # Show the plot
        plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the Courant number dataframe:")
max_courant_gdf


In [ ]:
# Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant)

max_face_shear_gdf = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Shear Stress")

In [ ]:
print("\nMesh Summary Output (Maximum Face Shear Stress:")
print(max_face_shear_gdf.attrs)

In [ ]:
max_face_shear_gdf

In [ ]:
# Plot max face shear stress

if generate_maps and not max_face_shear_gdf.empty:
    # Calculate centroids of the line geometries and extract coordinates
    max_face_shear_gdf['centroid'] = max_face_shear_gdf['geometry'].apply(lambda line: line.centroid)
    max_face_shear_gdf['x'] = max_face_shear_gdf['centroid'].apply(lambda point: point.x)
    max_face_shear_gdf['y'] = max_face_shear_gdf['centroid'].apply(lambda point: point.y)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_face_shear_gdf['x'], max_face_shear_gdf['y'],
                        c=max_face_shear_gdf['maximum_face_shear_stress'],
                        cmap='viridis',
                        s=10)

    # Customize the plot
    ax.set_title('Max Face Shear Stress per Face')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Face Shear Stress (PSF)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
# Get mesh summary output for Minimum Water Surface as geodataframe
summary_gdf_min_ws = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Water Surface")

In [ ]:
print("\nMesh Summary Output (Minimum Water Surface):")
summary_gdf_min_ws

In [ ]:
# Get mesh summary output for Minimum Face Velocity as geodataframe
summary_gdf_min_fv = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Face Velocity")

In [ ]:
print("\nMesh Summary Output (Minimum Face Velocity):")
summary_gdf_min_fv

In [ ]:
# Get mesh summary output for Cell Cumulative Iteration as geodataframe
summary_gdf_cum_iter = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Cell Cumulative Iteration")

In [ ]:
print("\nMesh Summary Output (Cell Cumulative Iteration):")
summary_gdf_cum_iter

In [ ]:
# Get mesh timeseries output as xarray
# The mesh name is part of the timeseries HDF path, so you must pass the mesh_name to retrieve it

# Get mesh areas from previous code cell
mesh_areas = HdfMesh.get_mesh_area_names(geom_hdf_path)


In [ ]:
mesh_areas

In [ ]:
# Use the first mesh area name to extract mesh timeseries output as xarray
timeseries_xr = HdfResultsMesh.get_mesh_timeseries(plan_hdf_path, mesh_areas[0], "Water Surface") # Use the first 2D flow area name for mesh_name

In [ ]:
timeseries_xr

In [ ]:
# Time Series Output Variables for Cells
# 
# Variable Name: Description
# Water Surface: Water surface elevation
# Depth: Water depth
# Velocity: Magnitude of velocity
# Velocity X: X-component of velocity
# Velocity Y: Y-component of velocity
# Froude Number: Froude number
# Courant Number: Courant number
# Shear Stress: Shear stress on the bed
# Bed Elevation: Elevation of the bed
# Precipitation Rate: Rate of precipitation
# Infiltration Rate: Rate of infiltration
# Evaporation Rate: Rate of evaporation
# Percolation Rate: Rate of percolation
# Groundwater Elevation: Elevation of groundwater
# Groundwater Depth: Depth to groundwater
# Groundwater Flow: Groundwater flow rate
# Groundwater Velocity: Magnitude of groundwater velocity
# Groundwater Velocity X: X-component of groundwater velocity
# Groundwater Velocity Y: Y-component of groundwater velocity
# 
# These variables are available for time series output at the cell level in 2D flow areas.


In [ ]:
# Get mesh cells timeseries output as xarray
cells_timeseries_xr = HdfResultsMesh.get_mesh_cells_timeseries(plan_hdf_path, mesh_areas[0])

In [ ]:
cells_timeseries_xr

In [ ]:
# Plot WSE Time Series Data (Random Cell ID) 
import matplotlib.pyplot as plt

if generate_plots:
    import numpy as np
    import random

    # Extract Water Surface data
    water_surface = cells_timeseries_xr[mesh_areas[0]]['Water Surface']

    # Get the time values
    time_values = water_surface.coords['time'].values

    # Pick a random cell_id
    random_cell_id = random.choice(water_surface.coords['cell_id'].values)

    # Extract the water surface elevation time series for the random cell
    wsel_timeseries = water_surface.sel(cell_id=random_cell_id)

    # Find the peak value and its index
    peak_value = wsel_timeseries.max().item()
    peak_index = wsel_timeseries.argmax().item()

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time_values, wsel_timeseries, label=f'Cell ID: {random_cell_id}')
    plt.scatter(time_values[peak_index], peak_value, color='red', s=100, zorder=5)
    plt.annotate(f'Peak: {peak_value:.2f} ft', 
                (time_values[peak_index], peak_value),
                xytext=(10, 10), textcoords='offset points',
                ha='left', va='bottom',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

    plt.title(f'Water Surface Elevation Time Series for Random Cell (ID: {random_cell_id})')
    plt.xlabel('Time')
    plt.ylabel('Water Surface Elevation (ft)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Log the plotting action
    logging.info(f"Plotted water surface elevation time series for random cell ID: {random_cell_id}")

    # Display the plot
    plt.show()

    # Print some statistics
    print(f"Statistics for Cell ID {random_cell_id}:")
    print(f"Minimum WSEL: {wsel_timeseries.min().item():.2f} ft")
    print(f"Maximum WSEL: {peak_value:.2f} ft")
    print(f"Mean WSEL: {wsel_timeseries.mean().item():.2f} ft")
    print(f"Time of peak: {time_values[peak_index]}")

In [ ]:
# Get mesh faces timeseries output as xarray
faces_timeseries_xr = HdfResultsMesh.get_mesh_faces_timeseries(plan_hdf_path, mesh_areas[0])

In [ ]:
faces_timeseries_xr

In [ ]:
# Plot Random Face Results and Label Peak, Plus Map View

if generate_maps:

    # Select a random valid face ID number
    random_face = np.random.randint(0, faces_timeseries_xr.sizes['face_id'])

    # Extract time series data for the selected face
    variable = 'face_velocity'  # We could also use 'face_flow'
    face_data = faces_timeseries_xr[variable].sel(face_id=random_face)

    # Find peak value and its corresponding time
    peak_value = face_data.max().item()
    peak_time = face_data.idxmax().values

    # Plot time series
    plt.figure(figsize=(12, 8))
    plt.plot(faces_timeseries_xr.time, face_data)
    plt.title(f'{variable.capitalize()} Time Series for Face {random_face}')
    plt.xlabel('Time')
    plt.ylabel(f'{variable.capitalize()} ({faces_timeseries_xr.attrs["units"]})')
    plt.grid(True)

    # Annotate the peak point
    plt.annotate(f'Peak: ({peak_time}, {peak_value:.2f})', 
                (peak_time, peak_value),
                xytext=(10, 10), textcoords='offset points',
                arrowprops=dict(arrowstyle="->"))

    # Check for negative values and label the minimum if present
    min_value = face_data.min().item()
    if min_value < 0:
        min_time = face_data.idxmin().values
        plt.annotate(f'Min: ({min_time}, {min_value:.2f})', 
                    (min_time, min_value),
                    xytext=(10, -10), textcoords='offset points',
                    arrowprops=dict(arrowstyle="->"))

    plt.tight_layout()
    plt.show()

    # Create map view plot
    fig, ax = plt.subplots(figsize=(12, 8))


    # Calculate mesh faces extents with 10% buffer
    faces_bounds = mesh_cell_faces_gdf.total_bounds
    x_min, y_min, x_max, y_max = faces_bounds
    buffer_x = (x_max - x_min) * 0.1
    buffer_y = (y_max - y_min) * 0.1
    plot_xlim = [x_min - buffer_x, x_max + buffer_x]
    plot_ylim = [y_min - buffer_y, y_max + buffer_y]

    # Set plot limits before adding terrain
    ax.set_xlim(plot_xlim)
    ax.set_ylim(plot_ylim)

    # Add the terrain TIFF to the map, clipped to our desired extent
    tiff_path = Path.cwd() / 'example_projects' / 'BaldEagleCrkMulti2D' / 'Terrain' / 'Terrain50.baldeagledem.tif'
    with rasterio.open(tiff_path) as src:
        show(src, ax=ax, cmap='terrain', alpha=0.5)
        
    # Reset the limits after terrain plot
    ax.set_xlim(plot_xlim)
    ax.set_ylim(plot_ylim)

    # Plot all faces in gray
    mesh_cell_faces_gdf.plot(ax=ax, color='lightgray', alpha=0.5, zorder=2)

    # Get the selected face geometry
    selected_face = mesh_cell_faces_gdf[mesh_cell_faces_gdf['face_id'] == random_face]

    # Highlight the selected face in red
    selected_face.plot(
        ax=ax, 
        color='red',
        linewidth=2,
        label=f'Selected Face (ID: {random_face})',
        zorder=3
    )

    # Get bounds of selected face for zoomed inset
    bounds = selected_face.geometry.bounds.iloc[0]
    x_center = (bounds.iloc[0] + bounds.iloc[2]) / 2
    y_center = (bounds.iloc[1] + bounds.iloc[3]) / 2
    buffer = max(bounds.iloc[2] - bounds.iloc[0], bounds.iloc[3] - bounds.iloc[1]) * 2

    # Create zoomed inset with a larger size, inside the map frame
    axins = inset_axes(ax, width="70%", height="70%", loc='lower right',
                    bbox_to_anchor=(0.65, 0.05, 0.35, 0.35),
                    bbox_transform=ax.transAxes)

    # Plot terrain and faces in inset
    with rasterio.open(tiff_path) as src:
        show(src, ax=axins, cmap='terrain', alpha=0.5)
        
    # Plot zoomed view in inset
    mesh_cell_faces_gdf.plot(ax=axins, color='lightgray', alpha=0.5, zorder=2)
    selected_face.plot(ax=axins, color='red', linewidth=2, zorder=3)

    # Set inset limits with slightly more context
    axins.set_xlim(x_center - buffer/1.5, x_center + buffer/1.5)
    axins.set_ylim(y_center - buffer/1.5, y_center + buffer/1.5)

    # Remove inset ticks for cleaner look
    axins.set_xticks([])
    axins.set_yticks([])

    # Add a border to the inset
    for spine in axins.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1.5)

    # Create connection lines between main plot and inset
    # Get the selected face centroid for connection point
    centroid = selected_face.geometry.centroid.iloc[0]
    con1 = ConnectionPatch(
        xyA=(centroid.x, centroid.y), coordsA=ax.transData,
        xyB=(0.02, 0.98), coordsB=axins.transAxes,
        arrowstyle="-", linestyle="--", color="gray", alpha=0.6
    )
    con2 = ConnectionPatch(
        xyA=(centroid.x, centroid.y), coordsA=ax.transData,
        xyB=(0.98, 0.02), coordsB=axins.transAxes,
        arrowstyle="-", linestyle="--", color="gray", alpha=0.6
    )

    ax.add_artist(con1)
    ax.add_artist(con2)

    # Add title and legend to main plot
    ax.set_title('Mesh Face Map View with Terrain')
    ax.legend()

    # Ensure equal aspect ratio while maintaining our desired extents
    ax.set_aspect('equal', adjustable='box')

    plt.tight_layout()
    plt.show()

    # Print summary information
    print(f"Random Face: {random_face}")
    print(f"Peak Value: {peak_value:.2f} {faces_timeseries_xr.attrs['units']} at {peak_time}")
    if min_value < 0:
        print(f"Minimum Value: {min_value:.2f} {faces_timeseries_xr.attrs['units']} at {min_time}")

    # Log the plotting action
    logging.info(f"Plotted mesh face time series and map view for random face ID: {random_face} with terrain")

In [ ]:
# Get meteorology precipitation attributes
meteo_precip_attrs = HdfPlan.get_plan_met_precip(plan_hdf_path)


In [ ]:
meteo_precip_attrs

In [ ]:
# Get results unsteady attributes
results_unsteady_attrs = HdfResultsPlan.get_unsteady_info(plan_hdf_path)


In [ ]:
results_unsteady_attrs

In [ ]:
# Get results unsteady summary attributes
results_unsteady_summary_attrs = HdfResultsPlan.get_unsteady_summary(plan_hdf_path)


In [ ]:
results_unsteady_summary_attrs

In [ ]:
# Get results volume accounting attributes
volume_accounting_attrs = HdfResultsPlan.get_volume_accounting(plan_hdf_path)

In [ ]:
volume_accounting_attrs

-----

In [ ]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np



def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"



try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

 



## Exploring HDF Datasets with HdfBase.get_dataset_info
This allows users to find HDF information that is not included in the ras-commander library.  Find the path in HDFView and set the group_path below to explore the HDF datasets and attributes.  Then, use the output to write your own function to extract the data.  

#### Get HDF Paths with Properties (For Exploring HDF Files)
HdfBase.get_dataset_info(plan_number, group_path="/")

For HDF datasets that are not supported by the RAS-Commadner library, provide the dataset path to HdfBase.get_dataset_info and provide the output to an LLM along with a relevent HDF* class(es) to generate new functions that extend the library's coverage.   